<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/Copy_of_OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/tools/OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OnDemandLoaderTool

Our `OnDemandLoaderTool` is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.

This tool takes in a `BaseReader` data loader, and when called will 1) load data, 2) index data, and 3) query the data.

In this walkthrough, we show how to use the `OnDemandLoaderTool` to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

In [2]:
%pip install llama-index-readers-wikipedia -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


In [3]:
!pip install llama-index -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 27.2 MB/s eta 0:00:00


In [4]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

In [5]:
!pip install wikipedia -q

  Preparing metadata (setup.py) ... done


In [6]:
reader = WikipediaReader()

In [7]:
tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

In [9]:
!pip install langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 16.5 MB/s eta 0:00:00


In [ ]:
#added by Frank Morales(FM) 22/02/2024
%pip install openai  --root-user-action=ignore -q
%pip install colab-env --upgrade --quiet --root-user-action=ignore -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
import warnings
warnings.filterwarnings('ignore')

import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()

In [ ]:
tool(["Montreal"], query_str="where is located the Montreal city?")

In [30]:
# run tool as langchain structured tool
lc_tool = tool.to_langchain_structured_tool(verbose=True)

In [67]:
from langchain.agents import AgentType, initialize_agent, load_tools
#from langchain_openai import ChatOpenAI, OpenAI
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

#llm = ChatOpenAI(temperature=0.0)
math_llm = OpenAI(temperature=0.0)

#llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)
llm = ChatOpenAI(temperature=0, model_name="gpt-4", streaming=True)

tools = load_tools(
    ["human", "llm-math"],
    llm=math_llm,
)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #agent="structured-chat-zero-shot-react-description",
    verbose=True,
)

agent = initialize_agent(
    [lc_tool],
    #tools,
    llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=True,
)

In [52]:
agent_chain.run("What's my friend Rafael's surname?")
# Answer with 'Reinaldo'



> Entering new AgentExecutor chain...
I need to ask a human for this information.
Action: human
Action Input: What is Rafael's surname?

What is Rafael's surname?
Reinaldo

Observation: Reinaldo
Thought:I now know Rafael's surname.
Final Answer: Rafael's surname is Reinaldo.

> Finished chain.


"Rafael's surname is Reinaldo."

In [48]:
#agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"},query_str="What's the arts and culture scene in montreal?")

agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"})




> Entering new AgentExecutor chain...
Thought: I should use the Wikipedia Tool to gather information about the arts and culture scene in Montreal.

Action:
```
{
  "action": "Wikipedia Tool",
  "action_input": {
    "pages": ["Montreal"],
    "query_str": "Arts and culture scene in Montreal"
  }
}
```
Observation: Montreal boasts a vibrant arts and culture scene, earning it the title of "Canada's Cultural Capital." The city is known for its diverse cultural communities, which have contributed to a unique local culture. Montreal is a hub for various forms of artistic expression including visual arts, theatre, dance, and music, with a strong tradition in jazz and rock music. The city hosts over 100 annual festivals, with highlights such as the Montreal International Jazz Festival, the largest of its kind globally, and the Just for Laughs comedy festival. Montreal's downtown area comes alive during the summer with cultural and social events, showcasing the city's dynamic cultural ecosys

"Montreal boasts a vibrant arts and culture scene, earning it the title of 'Canada's Cultural Capital.' The city is known for its diverse cultural communities, which have contributed to a unique local culture. Montreal is a hub for various forms of artistic expression including visual arts, theatre, dance, and music, with a strong tradition in jazz and rock music. The city hosts over 100 annual festivals, with highlights such as the Montreal International Jazz Festival, the largest of its kind globally, and the Just for Laughs comedy festival. Montreal's downtown area comes alive during the summer with cultural and social events, showcasing the city's dynamic cultural ecosystem. Additionally, Montreal is home to renowned cultural venues like the Place des Arts, which hosts a variety of performances ranging from classical music to avant-garde dance. The city's rich cultural heritage is further exemplified by its numerous churches, including impressive basilicas and historic landmarks. M

In [68]:
agent.run(input={"pages": "Montreal", "query_str": "What is the best restaurant in Montreal?"})



> Entering new AgentExecutor chain...
Thought: The user is asking for the best restaurant in Montreal. I will use the Wikipedia Tool to find information about Montreal's top restaurants.
Action:
```
{
  "action": "Wikipedia Tool",
  "action_input": {
    "pages": ["Montreal"],
    "query_str": "best restaurant"
  }
}
```
Observation: The best restaurant in Montreal is a subjective matter and can vary depending on personal preferences. Montreal is known for its diverse culinary scene, offering a wide range of dining options from traditional French cuisine to innovative fusion dishes. Visitors and locals alike often recommend exploring the city's vibrant neighborhoods like Old Montreal, Plateau-Mont-Royal, and Mile End to discover hidden gems and popular dining spots. It is advisable to check online reviews, ask locals for recommendations, and explore different types of cuisine to find the best restaurant that suits your taste preferences.
Thought:The Wikipedia tool provided a general 

"The best restaurant in Montreal can vary depending on personal preferences. Montreal is known for its diverse culinary scene, offering a wide range of dining options from traditional French cuisine to innovative fusion dishes. It's recommended to explore the city's vibrant neighborhoods like Old Montreal, Plateau-Mont-Royal, and Mile End to discover popular dining spots. Checking online reviews and asking locals for recommendations can also help you find a restaurant that suits your taste preferences."